# Q1

The data is given as below

In [2]:
using SparseArrays

m = 5
n = 5
rowindices = [3, 1, 1, 4, 5]
colindices = [1, 2, 3, 3, 4]
data = [0.799, 0.942, 0.848, 0.164, 0.637]

sp = sparse(rowindices, colindices, data, m, n)

5×5 SparseMatrixCSC{Float64, Int64} with 5 stored entries:
  ⋅     0.942  0.848   ⋅      ⋅ 
  ⋅      ⋅      ⋅      ⋅      ⋅ 
 0.799   ⋅      ⋅      ⋅      ⋅ 
  ⋅      ⋅     0.164   ⋅      ⋅ 
  ⋅      ⋅      ⋅     0.637   ⋅ 

In [3]:
println(sp.colptr)
println(sp.rowval)
println(sp.nzval)

[1, 2, 3, 5, 6, 6]
[3, 1, 1, 4, 5]
[0.799, 0.942, 0.848, 0.164, 0.637]


# Q2(1)

In [4]:
function my_spv(sp, v)
    @assert sp.n == size(v, 1)
    n = sp.n
    m = sp.m
    spv = zeros(Float64, m)
    sp_val = sp.nzval
    sp_row = sp.rowval
    for i in 1:n
        for j in nzrange(sp, i)
            k = sp_row[j]
            spv[k] += sp_val[j] * v[i]
        end
    end
    return spv
end

my_spv (generic function with 1 method)

In [5]:
using SparseArrays, Test

@testset "sparse matrix - vector multiplication" begin
	for k = 1:100
		m, n = rand(1:100, 2)
		density = rand()
		sp = sprand(m, n, density)
		v = randn(n)
        @test Matrix(sp) * v ≈ my_spv(sp, v)
	end
end

Test Summary:                         | Pass  Total  Time
sparse matrix - vector multiplication |  100    100  0.5s


Test.DefaultTestSet("sparse matrix - vector multiplication", Any[], 100, false, false, true, 1.679150306473528e9, 1.679150306969063e9)

# Q2(2)

In [32]:
# here I use code in class for lanczos process
function lanczos(A, q1::AbstractVector{T}; abstol, maxiter) where T
	# normalize the input vector
	q1 = normalize(q1)
	# the first iteration
	q = [q1]
	Aq1 = A * q1
	α = [q1' * Aq1]
	rk = Aq1 .- α[1] .* q1
	β = [norm(rk)]
	for k = 2:min(length(q1), maxiter)
		# the k-th orthonormal vector in Q
		push!(q, rk ./ β[k-1])
		Aqk = A * q[k]
		# compute the diagonal element as αₖ = qₖᵀ A qₖ
		push!(α, q[k]' * Aqk)
		rk = Aqk .- α[k] .* q[k] .- β[k-1] * q[k-1]
		# compute the off-diagonal element as βₖ = |rₖ|
		nrk = norm(rk)
		# break if βₖ is smaller than abstol or the maximum number of iteration is reached
		if abs(nrk) < abstol || k == length(q1)
			break
		end
		push!(β, nrk)
	end
	# returns T and Q
	return SymTridiagonal(α, β), cat(q1, q[2:end]...; dims=2)
end

lanczos (generic function with 1 method)

In [38]:
using LinearAlgebra

function restarting(T, Q)
    eigen_T = eigen(T)
    eigen_val = eigen_T.values
    eigen_vec = eigen_T.vectors
    
    λ = eigen_val[end]
    u_1 = eigen_vec[:, end]
    q_1 = Q * u_1
    return λ, q_1
end

restarting (generic function with 1 method)

In [39]:
function lanczos_restarting(A, q_1, abstol, maxiter, error_cut, maxiter_re)
    # first iteration
    T, Q = lanczos(A, q_1; abstol, maxiter)
    λ, q_1 = restarting(T, Q)

    λ_pre = λ
    turns = 1
    error = 1 + error_cut

    while abs(error) > error && maxiter_re < turns
        T, Q = lanczos(A, q_1; abstol, maxiter)
        λ, q_1 = restarting(T, Q)

        error = λ_pre - λ
        λ_pre = λ
        turns += 1
    end
    return λ, q_1
end

lanczos_restarting (generic function with 1 method)

In [40]:
@testset "lanczos_restarting test" begin
	for k = 1:100
		n = rand(1:100)
		abstol = 1e-10
		maxiter = n
		error_cut = 1e-10
		maxiter_re = n

		A = rand(Float64, (n, n))
		A = Symmetric(A)
		q_1 = rand(Float64, n)
		λ, q_1_norm = lanczos_restarting(A, q_1, abstol, maxiter, error_cut, maxiter_re)
        @test abs(eigvals(A)[end] - λ) < abstol
	end
end

Test Summary:           | Pass  Total  Time
lanczos_restarting test |  100    100  0.1s


Test.DefaultTestSet("lanczos_restarting test", Any[], 100, false, false, true, 1.679151172858877e9, 1.67915117294241e9)